In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from tqdm import tqdm

In [15]:
training = pd.read_csv('C:/Users/hangu/Creative Cloud Files/0. Data Science study/Datasets/Home Credit default risk/application_train.csv')
testing = pd.read_csv('C:/Users/hangu/Creative Cloud Files/0. Data Science study/Datasets/Home Credit default risk/application_test.csv')
bureau = pd.read_csv('C:/Users/hangu/Creative Cloud Files/0. Data Science study/Datasets/Home Credit default risk/bureau.csv')
bureau_balance = pd.read_csv('C:/Users/hangu/Creative Cloud Files/0. Data Science study/Datasets/Home Credit default risk/bureau_balance.csv')  # 'STATUS' meaning?

In [37]:
POS_CASH = pd.read_csv('C:/Users/hangu/Creative Cloud Files/0. Data Science study/Datasets/Home Credit default risk/POS_CASH_balance.csv')


In [40]:
POS_CASH['SK_ID_PREV'].value_counts()

SK_ID_PREV
1856103    96
2706683    96
1617536    96
1364606    96
1057553    96
           ..
1922777     1
2660098     1
1364218     1
1077449     1
1191779     1
Name: count, Length: 936325, dtype: int64

In [41]:
POS_CASH['SK_ID_CURR'].value_counts()

SK_ID_CURR
265042    295
172612    247
309133    246
127659    245
185185    245
         ... 
406557      1
240277      1
177557      1
251697      1
144536      1
Name: count, Length: 337252, dtype: int64

In [49]:
sample = POS_CASH[POS_CASH['SK_ID_CURR']==265042]
sample[sample['SK_ID_PREV']==1776290].sort_values(by='MONTHS_BALANCE').head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
5292916,1776290,265042,-33,36.0,36.0,Active,0,0
5151868,1776290,265042,-32,36.0,35.0,Active,0,0
4886018,1776290,265042,-31,36.0,34.0,Active,0,0
3762157,1776290,265042,-30,36.0,33.0,Active,0,0
5226689,1776290,265042,-29,36.0,32.0,Active,0,0


In [34]:
print(bureau_balance.columns, "\n")
pd.DataFrame(bureau_balance['SK_ID_BUREAU'].value_counts()).transpose()     # Number of Months in balance = count - 1

Index(['SK_ID_BUREAU', 'MONTHS_BALANCE', 'STATUS'], dtype='object') 



SK_ID_BUREAU,5645521,6733619,6176606,6321834,6356432,6356400,6243196,6356352,6356351,6765607,...,5091397,5091393,5298428,5298481,6212591,6212548,5819605,5626301,5626300,6803532
count,97,97,97,97,97,97,97,97,97,97,...,1,1,1,1,1,1,1,1,1,1


In [30]:
print(bureau.columns, "\n")
print(bureau['SK_ID_BUREAU'].value_counts())
pd.DataFrame(bureau['SK_ID_CURR'].value_counts()).transpose()


Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY'],
      dtype='object') 

SK_ID_BUREAU
5714462    1
6084809    1
6084794    1
6084795    1
6084796    1
          ..
6060713    1
6060715    1
6060720    1
6060747    1
6745061    1
Name: count, Length: 1048575, dtype: int64


SK_ID_CURR,120860,169704,318065,251643,425396,295809,177014,280155,252366,218175,...,304748,430469,160781,334642,127041,234522,180076,171655,258946,399573
count,116,81,78,61,60,59,56,55,55,55,...,1,1,1,1,1,1,1,1,1,1


In [5]:
training

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
testing

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
